In [ ]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [16]:
from tensorflow import keras
from tensorflow.keras import layers,models,optimizers

model=models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(180,180,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512,activation="relu"))
model.add(layers.Dense(1,activation="sigmoid"))



In [17]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 178, 178, 32)      896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 89, 89, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 87, 87, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 43, 43, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 41, 41, 128)       73856     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 18, 18, 128)      

In [18]:
model.compile(loss="binary_crossentropy",
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=["accuracy"])

# birincil yöntem 

In [33]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dir="/media/serkancam/yedek/Calismalar/python/derin_ogrenme/kodlarim/dogs-vs-cats/cats_vs_dogs_small/train"
validation_dir="/media/serkancam/yedek/Calismalar/python/derin_ogrenme/kodlarim/dogs-vs-cats/cats_vs_dogs_small/validation"
train_datagen =  ImageDataGenerator(rescale=1./255,
                                    # rotation_range=40,
                                    # width_shift_range=0.2,
                                    # height_shift_range=0.2,
                                    # shear_range=0.2,
                                    # zoom_range=0.2,
                                    # horizontal_flip=True                                    
                                    )
test_datagen =  ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
                                train_dir,
                                target_size=(180,180),
                                batch_size=32,
                                class_mode="binary"
                                )
validation_generator=train_datagen.flow_from_directory(
                                validation_dir,
                                target_size=(180,180),
                                batch_size=32,
                                class_mode="binary"
                                )

for data_batch,labels_batch in train_generator:
    print("data batch shape:",data_batch.shape)
    print("labels batch shape:",labels_batch.shape)
    break

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
data batch shape: (32, 180, 180, 3)
labels batch shape: (32,)


In [ ]:
print(len(train_generator))
print(train_generator.batch_size)
print(len(validation_generator))


63
32
32


In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=callbacks)

In [ ]:
history=model.fit(
    train_generator,
    # steps_per_epoch=63,
    epochs=30,
    validation_data=validation_generator,
    # validation_steps=32,
    
)

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["acc"]
val_accuracy = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()